# Motion Compensated Discrete Wavelet Transform (MCDWT)

MCDWT is a [video] decorrelator and [visual] information organizer. The
input sequence of pixels_ are [decorrelated] in the [time] and in the
[spatial domains]. The output sequence of [coefficients] have a
smaller [entropy] than the original pixels, and the information is
represented by [resolution levels].

Temporal decorrelation is provided by [MC (Motion Compensation)], where the
[prediction images] are generated with an algorithm that uses only the
information available at the [decoder]. This means that the motion
information used in the predictions do not need to be sent to the
decoder.

Spatial decorrelation is performed by the [analysis filter] used in the
[2D-DWT].

[video]: https://en.wikipedia.org/wiki/Video
[visual]: https://en.wikipedia.org/wiki/Visual_perception
[pixels]: https://en.wikipedia.org/wiki/Pixel
[decorrelated]: https://en.wikipedia.org/wiki/Decorrelation
[time]: https://en.wikipedia.org/wiki/Time_domain
[spatial]: https://www.quora.com/What-is-spatial-domain-in-image-processing
[coefficients]: https://www.quora.com/What-is-spatial-domain-in-image-processing
[entropy]: https://en.wikipedia.org/wiki/Entropy
[resolution levels]: https://en.wikipedia.org/wiki/Image_resolution
[MC (Motion Compensation)]: https://en.wikipedia.org/wiki/Motion_compensation
[decoder]: https://en.wikipedia.org/wiki/Decoder
[prediction images]: https://en.wikipedia.org/wiki/Decoder
[analysis filter]: https://en.wikipedia.org/wiki/Digital_filter#Analysis_techniques
[2D-DWT]: https://en.wikipedia.org/wiki/Discrete_wavelet_transform
[EBCOT]: http://nptel.ac.in/courses/117105083/pdf/ssg_m5l15.pdf

## Video [scalability]

MCDWT inputs a [video] and outputs a video, in a way that when using
only a portion of the data of the transformed video, a video with a
lower [temporal resolution], lower [spatial resolution] or/and lower
[quality] can be generated.

If all the transformed data is used, then the original video is
obtained (MCDWT is a lossless transform). The video output has exactly
the same number of elements than the input video (for example, no
extra motion fields are produced). At this moment, we will focuse only
on spatial [scalability].

[temporal resolution]: https://en.wikipedia.org/wiki/Temporal_resolution
[spatial resolution]: https://en.wikipedia.org/wiki/Image_resolution#Spatial_resolution
[quality]: https://en.wikipedia.org/wiki/Compression_artifact
[scalability]: http://inst.eecs.berkeley.edu/~ee290t/sp04/lectures/videowavelet_UCB1-3.pdf
[video]: https://en.wikipedia.org/wiki/Video

## Video transformation alternatives

To obtain a multiresolution version or a video, the [DWT] (Discrete
Wavelet Transform) can be applied along temporal (t) and
spatial domains (2D). At this point, two alternatives
arise: (1) a t+2D transform or (2) a 2D+t
transform. In a t+2D transform, the video is first analyzed
over the time domain and next, over the space domain. A 2D+t
transform does just the opposite.

[DWT]: https://en.wikipedia.org/wiki/Discrete_wavelet_transform

Each choice has a number of *pros* and *cons*. For example, in a
t+2D transform we can apply directly any image predictor based
on motion estimation because the input is a normal video. However, if
we implement a 2D+t transform, the input to the motion
estimator is a sequence of images in the DWT domain. [The overwhelming
majority of DWT's][Friendly Guide] are not [shift invariant], which basically means
$\text{DWT}(s[t]) \neq \text{DWT}(s[t+x])$, where $x$ is a
displacement of the $s[t]$ along the time
domain. Therefore, motion estimators which compare pixel values will
not work on the DWT domain. On the other hand, if we want to provide
true spatial scalability (processing only those spatial resolutions
(scales) necessary to get a spatially scaled of our video), a
t+2D transformed video is not suitable because the first step
of the forward transform (t) should be reversed at full
resolution in the backward transform (as the forward transform did).

[shift invariant]: http://www.polyvalens.com/blog/wavelets/theory
[Friendly Guide]: http://www.polyvalens.com/blog/wavelets/theory

## Wavelet and pyramid domains

Indeed, the DWT allows to get a scalable representation of a image and by extension, of a video if we apply the DWT on all the images of the video. However, this can be also done with [Gaussian and Laplacian pyramids][Laplacian Pyramids]. Image pyramids are interesting because they are shift invariant and therefore, one can operate within the scales as they are *normal* images. Unfortunately, as a consecuence of pyramids representations are not critically sampled, they need more picture elements than in [Wavelet pyramids][Wavelet Pyramids] and this is a drawback when compressing. Luckily, it is very fast to convert a Laplacian pyramid representation into it DWT equivalent representation, and viceversa. For this reason, even if we use the Wavelet pyramids to work with our images, we can suppose at any moment that we are working with the Laplacian pyramid of those images.

[Laplacian Pyramids]: https://en.wikipedia.org/wiki/Pyramid_(image_processing)
[Wavelet Pyranids]: http://www.vtvt.ece.vt.edu/research/references/video/DCT_Video_Compression/Zhang92a.pdf

## The 's'-levels 2D Discrete Wavelet Transform

A [2D-DWT][2D-DWT] (2 Dimensions - Discrete Wavelet Transform) generates a scalable representation of an image and by extension, of a video if we apply the DWT on all the images of the video. This is done, for example, in [the JPEG2000 image and video compression standard][J2K]. Notice that only the spatial redundancy is exploited. All the temporal redundancy is still in the video.

[J2K]: https://en.wikipedia.org/wiki/JPEG_2000
[2D-DWT]: https://en.wikipedia.org/wiki/Discrete_wavelet_transform

### Input

A sequence `V` of `n` images:

```
                                                         x
+---------------+  +---------------+     +---------------+
|               |  |               |     |            |  |
|               |  |               |   y |----------- O <---- V[n-1][y][x]
|               |  |               | ... |               |
|               |  |               |     |               |
|               |  |               |     |               |
|               |  |               |     |               |
+---------------+  +---------------+     +---------------+
      V[0]               V[1]                 V[n-1]
```

### Output

A sequence `S` of `n` "pyramids". For example, a 2-levels 2D-DWT looks like:

```
+---+---+-------+  +---+---+-------+     +---+---+-------+
|LL2|HL2|       |  |   |   |       |     |   |   |       |
+---+---+  HL1  |  +---+---+       |     +---+---+       |
|LH2|HH2|       |  |   |   |       |     |   |   |       |
+---+---+-------+  +---+---+-------+ ... +---+---+-------+
|       |       |  |       |       |     |       |       |
|  LH1  |  HH1  |  |       |       |     |       |       |
|       |       |  |       |       |     |       |       |        
+-------+-------+  +-------+-------+     +-------+-------+
S[0]               S[1]                  S[2]
```
where `L` and `H` stands for *low-pass filtered* and *high-pass filtered*, respectively. The integer > 1 that follows these letters represents the subband level. For the sake of simplicity, we will denote the subbands `{LH, HL, HH}` as only `H`, and `LL` as only `L`. 

### Algorithm
```pytho
for image in V:
  2D_DWT(image) # In place
S = V # Pointer copy
```